In [1]:
import numpy as np
import scipy.io
from multiprocessing import Pool
from functools import partial
import pickle
import scipy
import scipy.optimize
from sklearn.metrics import *
import scipy.stats as stat
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
import pandas as pd

## Import fit parameters

In [56]:
fits_single = pickle.load(open('fit_single_bound_with_transfer.pickle', 'rb'))
fits_dual = pickle.load(open('fit_dual_bound_with_transfer.pickle', 'rb'))

## Import single and dual state model functions

In [2]:
from dual_model_with_transfer import dual_model_sudden, dual_model_gradual
from single_model_with_transfer import model_sudden, model_gradual

## Generate Data

In [3]:
model_errors_dual = np.zeros((60, 704))
model_errors_single = np.zeros((60, 704))
for participant in range(60):
    if participant%4 == 0 or participant%4 == 1:
        model_errors_dual[participant] = dual_model_sudden(704, fits_dual[participant][0], fits_dual[participant][1], fits_dual[participant][2], fits_dual[participant][3])[0]
        model_errors_single[participant] = model_sudden(704, fits_single[participant][0], fits_single[participant][1])[0]

    else:
        model_errors_dual[participant] = dual_model_gradual(704, fits_dual[participant][0], fits_dual[participant][1], fits_dual[participant][2], fits_dual[participant][3])[0]
        model_errors_single[participant] = model_gradual(704, fits_single[participant][0], fits_single[participant][1])[0]
        

NameError: name 'fits_dual' is not defined

## Save generated data

In [59]:
generated_by_dual = np.zeros((60, 12, 64))
generated_by_dual[:, 1:] = np.reshape(model_errors_dual, (60, 11, 64))

generated_by_single = np.zeros((60, 12, 64))
generated_by_single[:, 1:] = np.reshape(model_errors_single, (60, 11, 64))

pickle.dump(generated_by_dual, open('generated_by_dual.pickle', 'wb'))
pickle.dump(generated_by_single, open('generated_by_single.pickle', 'wb'))


# After Model Recovery Fits

### Read Recovery fits

In [4]:
fits_dual_GBD = pickle.load(open('fit_dual_bound_with_transfer_GBD.pickle', 'rb'))
fits_dual_GBS = pickle.load(open('fit_dual_bound_with_transfer_GBS.pickle', 'rb'))
fits_single_GBD = pickle.load(open('fit_single_bound_with_transfer_GBD.pickle', 'rb'))
fits_single_GBS = pickle.load(open('fit_single_bound_with_transfer_GBS.pickle', 'rb'))

fits_single_model_recovery = pickle.load(open('params/fit_single_bound_with_transfer_model_recovery.pickle', 'rb'))
fits_dual_model_recovery = pickle.load(open('params/fit_dual_bound_with_transfer_model_recovery.pickle', 'rb'))


In [13]:
AIC_dual_GBD = 2*4 + np.log(fits_dual_GBD[:, -1])
AIC_dual_GBS = 2*4 + np.log(fits_dual_GBS[:, -1])
AIC_single_GBD = 2*2 + np.log(fits_single_GBD[:, -1])
AIC_single_GBS = 2*2 + np.log(fits_single_GBS[:, -1])

BIC_dual_GBD = np.log(704)*4 + np.log(fits_dual_GBD[:, -1])
BIC_dual_GBS = np.log(704)*4 + np.log(fits_dual_GBS[:, -1])
BIC_single_GBD = np.log(704)*2 + np.log(fits_single_GBD[:, -1])
BIC_single_GBS = np.log(704)*2 + np.log(fits_single_GBS[:, -1])


AIC_smr = 2*2 + np.log(fits_single_model_recovery[:, -1])
AIC_dmr = 2*4 + np.log(fits_dual_model_recovery[:, -1])


In [11]:
conf_matrix = np.zeros((2, 2))
conf_matrix[0, 0] = np.sum(AIC_dual_GBD < AIC_single_GBD)
conf_matrix[0, 1] = np.sum(AIC_dual_GBD > AIC_single_GBD)
conf_matrix[1, 0] = np.sum(AIC_dual_GBS < AIC_single_GBS)
conf_matrix[1, 1] = np.sum(AIC_dual_GBS > AIC_single_GBS)

In [12]:
conf_matrix

array([[53.,  7.],
       [ 5., 55.]])

In [14]:
conf_matrix = np.zeros((2, 2))
conf_matrix[0, 0] = np.sum(BIC_dual_GBD < BIC_single_GBD)
conf_matrix[0, 1] = np.sum(BIC_dual_GBD > BIC_single_GBD)
conf_matrix[1, 0] = np.sum(BIC_dual_GBS < BIC_single_GBS)
conf_matrix[1, 1] = np.sum(BIC_dual_GBS > BIC_single_GBS)

In [15]:
conf_matrix

array([[46., 14.],
       [ 0., 60.]])